<a href="https://colab.research.google.com/github/drfperez/DeepPurpose/blob/main/DeepPurpose5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdkit
!pip install deeppurpose
!pip install descriptastorus
!pip install gradio
import numpy as np
import pandas as pd
from DeepPurpose import utils
from DeepPurpose import DTI as models
import gradio

model_binding = models.model_pretrained(model='MPNN_CNN_BindingDB')
model_kiba = models.model_pretrained(model='MPNN_CNN_KIBA')
model_davis = models.model_pretrained(model='MPNN_CNN_DAVIS')

def DTI_pred(data, drugs, targets):
    if data == 'BindingDB':
        model = model_binding
    elif data == 'KIBA':
        model = model_kiba
    elif data == 'DAVIS':
        model = model_davis

    # Process the drug and target data
    X_pred = utils.data_process(X_drug=drugs, X_target=targets, y=[0],
                                drug_encoding='MPNN', target_encoding='CNN',
                                split_method='no_split')
    # Predict the affinity for the drug and target pairs
    y_pred = model.predict(X_pred)

    # Convert predictions to list of strings
    predictions = [str(y) for y in y_pred]

    # Return the predictions
    return predictions

gradio.Interface(DTI_pred,
                 [gradio.Dropdown(label="Training Dataset", choices=['BindingDB', 'DAVIS', 'KIBA']),
                  gradio.Textbox(lines=5, label="Drug SMILES (separated by comma)"),
                  gradio.Textbox(lines=5, label="Target Amino Acid Sequence (separated by comma)")],
                 gradio.Textbox(label="Predicted Affinities (comma separated)")).launch(share=True)